<a href="https://colab.research.google.com/github/K-107/Ventilator_Pressure_Prediction/blob/main/test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
% cd /content/drive/MyDrive/Ventilator_Pressure_Prediction

/content/drive/MyDrive/Ventilator_Pressure_Prediction


In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv('train.csv')

In [7]:
df

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987
...,...,...,...,...,...,...,...,...
6035995,6035996,125749,50,10,2.504603,1.489714,1,3.869032
6035996,6035997,125749,50,10,2.537961,1.488497,1,3.869032
6035997,6035998,125749,50,10,2.571408,1.558978,1,3.798729
6035998,6035999,125749,50,10,2.604744,1.272663,1,4.079938


In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)
df_scaled = pd.DataFrame(data=scaled, columns=df.columns)
df_scaled

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,0.000000e+00,0.0,0.333333,1.0,0.000000,0.000833,0.0,0.115911
1,1.656727e-07,0.0,0.333333,1.0,0.011457,0.183830,0.0,0.116965
2,3.313453e-07,0.0,0.333333,1.0,0.022986,0.225093,0.0,0.146470
3,4.970180e-07,0.0,0.333333,1.0,0.034571,0.228088,0.0,0.204426
4,6.626906e-07,0.0,0.333333,1.0,0.046219,0.253559,0.0,0.211802
...,...,...,...,...,...,...,...,...
6035995,9.999993e-01,1.0,1.000000,0.0,0.852707,0.014897,1.0,0.086407
6035996,9.999995e-01,1.0,1.000000,0.0,0.864064,0.014885,1.0,0.086407
6035997,9.999997e-01,1.0,1.000000,0.0,0.875451,0.015590,1.0,0.085353
6035998,9.999998e-01,1.0,1.000000,0.0,0.886801,0.012727,1.0,0.089568


In [9]:
X = df_scaled.drop(['pressure'], axis=1)
y = df_scaled['pressure']

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

In [14]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

xgb = XGBRegressor(n_jobs=-1, random_state=20)
params = {'n_estimators':[400], 'max_depth':[2], 'learning_rate':[0.01]}
grid = GridSearchCV(xgb, param_grid=params, n_jobs=-1, cv=3)
grid.fit(X_train, y_train)
print('최적 하이퍼 파라미터: ', grid.best_params_)
best_grid = grid.best_estimator_
pred = best_grid.predict(X_test)

[03:25:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:25:46] WARNING: /workspace/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
최적 하이퍼 파라미터:  {'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 400}


In [18]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test,pred)

0.04633149570830192

In [20]:
df.describe()

,id,breath_id,R,C,time_step,u_in,u_out,pressure
count,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06,6.036000e+06
mean,3.018000e+06,6.283886e+04,2.703618e+01,2.608072e+01,1.307225e+00,7.321615e+00,6.204493e-01,1.122041e+01
std,1.742443e+06,3.633526e+04,1.959549e+01,1.715231e+01,7.659778e-01,1.343470e+01,4.852752e-01,8.109703e+00
min,1.000000e+00,1.000000e+00,5.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-1.895744e+00
25%,1.509001e+06,3.137700e+04,5.000000e+00,1.000000e+01,6.428995e-01,3.936623e-01,0.000000e+00,6.329607e+00
50%,3.018000e+06,6.276550e+04,2.000000e+01,2.000000e+01,1.308123e+00,4.386146e+00,1.000000e+00,7.032628e+00
75%,4.527000e+06,9.430100e+04,5.000000e+01,5.000000e+01,1.965502e+00,4.983895e+00,1.000000e+00,1.364103e+01
max,6.036000e+06,1.257490e+05,5.000000e+01,5.000000e+01,2.937238e+00,1.000000e+02,1.000000e+00,6.482099e+01


In [21]:
# minmax 복원
0.04633149570830192*(64.82099+1.895744)-1.895744

1.19534207499292